<a href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/tf2_keras_howtowrite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Digital Advantage - Deep Insider.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 「TensorFlow 2.0最新の書き方入門（全3回）」 ― 連載『TensorFlow 2.0＋Keras（ tf.keras）入門』のノートブック（2）

<table valign="middle">
  <td>
    <a target="_blank" href="https://www.atmarkit.co.jp/ait/subtop/features/di/tf2keras_index.html"> <img src="https://re.deepinsider.jp/img/ml-logo/manabu.svg"/>Deep Insiderで記事を読む</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/tf2_keras_howtowrite.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/gcolab.svg" />Google Colabで実行する</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/DeepInsider/playground-data/blob/master/docs/articles/tf2_keras_howtowrite.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/github.svg" />GitHubでソースコードを見る</a>
  </td>
</table>

※上から順に実行してください。上のコードで実行したものを再利用しているところがあるため、すべて実行しないとエラーになるコードがあります。  
　すべてのコードを一括実行したい場合は、メニューバーから［ランタイム］－［すべてのセルを実行］をクリックしてください（※TensofFlowのバージョン2.xでない場合、エラーになります）。

※「Python 3」を利用してください。  
　Python 3を利用するには、メニューバーから［ランタイム］－［ランタイムのタイプを変更］を選択すると表示される［ノートブックの設定］ダイアログの、［ランタイムのタイプ］欄で「Python 3」に選択し、その右下にある［保存］ボタンをクリックしてください。

# 第4回　知ってる？！ TensorFlow 2.0最新の書き方入門（初中級者向け）

## ■TensorFlowにおける、3種類／6通りのモデルの書き方

TensorFlow 2.0のモデルの書き方には下記の6通りある。本稿ではこの書き方を説明する。

**【Sequentialモデル】**（第4回）
- （1）`tf.keras.models.Sequential`クラスの**コンストラクター利用**
- （2）`tf.keras.models.Sequential`オブジェクトの`add`**メソッドで追加**

**【Functional API】**（第4回）
- （3）`tf.keras.Model`クラスの**コンストラクター利用**

**【Subclassingモデル】**（第5回と第6回）
- （4）`tf.keras.Model`クラスの**サブクラス化**
- （5）`TensorFlow低水準API`で**カスタム実装**

**【作成済みEstimators】**（第6回）
- （6）`tf.estimator.Estimator`が提供する**作成済みモデル**（**Pre-made Estimators**）の利用

### ●TensorFlow 2.0最新の書き方入門の記事構成（全3回）について

- [「第4回　知ってる？！ TensorFlow 2.0最新の書き方入門（初中級者向け）](https://www.atmarkit.co.jp/ait/articles/2002/27/news021.html)」……（1）～（3）
- 「第5回　お勧めの、TensorFlow 2.0最新の書き方入門（エキスパート向け）」……（4）
- 「第6回　カスタマイズするための、TensorFlow 2.0最新の書き方入門（エキスパート向け）」……（5）～（6）

### ●TensorFlowのエコシステム／機能構成図


![TensorFlow全体の機能構成図](https://github.com/DeepInsider/playground-data/raw/master/docs/articles/tensorflow-ecosystem-2.0.png)

## ■コードの書き方を実行するための準備

In [0]:
# Google Colabで最新の2.xを使う場合、2.xに切り替える（Colab専用）
%tensorflow_version 2.x

### ●前提条件

### 【チェック】Pythonバージョン（※3系を使うこと）
Colabにインストール済みのものを使う。もし2系になっている場合は、メニューバーの［ランタイム］－［ランタイムのタイプを変更］をクリックして切り替えてほしい。

In [0]:
import sys
print('Python', sys.version)
# Python 3.6.9 (default, Nov  7 2019, 10:44:02)  …… などと表示される

### 【チェック】TensorFlowバージョン（※2系を使うこと）
基本的にはColabにインストール済みのものを使う。もし2系になっている場合は、リスト4-0を実行してバージョン2.0を使う。

In [0]:
import tensorflow as tf
print('TensorFlow', tf.__version__)
# TensorFlow 1.15.0 ……などと表示される

#### リスト0-1　［オプション］ライブラリ「TensorFlow」最新バージョンのインストール

※Google Colabでを使う場合は、「%tensorflow_version 2.x」というマジックコマンドを使用してください。  
　「TensorFlow 2.x selected.」と表示されればOKです。
　「TensorFlow is already loaded. Please restart the runtime to change versions.」と表示される場合は、メニューバーから［ランタイム］－［ランタイムを再起動］を実行した上で、再度マジックコマンドの実行結果を確かめてください。

In [0]:
# Google Colabで最新の2.xを使う場合（Colab専用）
%tensorflow_version 2.x

# 最新バージョンにアップグレードする場合
!pip install --upgrade tensorflow

# バージョンを明示してアップグレードする場合
#!pip install --upgrade tensorflow===2.1.0

# 最新バージョンをインストールする場合
#!pip install tensorflow

# バージョンを明示してインストールする場合
#!pip install tensorflow===2.1.0

### ［オプション］【チェック】TensorFlowバージョン（※インストール後の確認）
バージョン2.xになっているか再度チェックする。

In [0]:
import tensorflow as tf
print('TensorFlow', tf.__version__)
# TensorFlow 2.1.0 ……などと表示される

### ●データについて

「[第1回　初めてのニューラルネットワーク実装、まずは準備をしよう ― 仕組み理解×初実装（前編）：TensorFlow 2＋Keras（tf.keras）入門 - ＠IT](https://www.atmarkit.co.jp/ait/articles/1909/19/news026.html)」の記事と同じように、シンプルな座標点データを生成して使う。使い方は、前述の記事を参照してほしい。

なお、座標点データは、「[ニューラルネットワーク Playground - Deep Insider](https://deepinsider.github.io/playground/)」（以下、Playground）と同じ生成仕様となっている。

#### リスト0-x　座標点データの生成

In [0]:
# 座標点データを生成するライブラリのインストール
!pip install playground-data

In [0]:
# playground-dataライブラリのplygdataパッケージを「pg」という別名でインポート
import plygdata as pg

# 設定値を定数として定義
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyCircleData # 問題種別：「分類（Classification）」、データ種別：「円（CircleData）」を選択
TRAINING_DATA_RATIO = 0.5  # データの何％を訓練【Training】用に？ (残りは精度検証【Validation】用) ： 50％
DATA_NOISE = 0.0           # ノイズ： 0％

# 定義済みの定数を引数に指定して、データを生成する
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

# データを「訓練用」と「精度検証用」を指定の比率で分割し、さらにそれぞれを「データ（X）」と「教師ラベル（y）」に分ける
X_train, y_train, X_valid, y_valid = pg.split_data(data_list, training_size=TRAINING_DATA_RATIO)

# データ分割後の各変数の内容例として、それぞれ5件ずつ出力（※出力内容は実行ごとに異なる）
print('X_train:'); print(X_train[:5]) # [[-0.07940614  1.15175421], ……]
print('y_train:'); print(y_train[:5]) # [[ 1.], ……  [-1.]]
print('X_valid:'); print(X_valid[:5]) # [[ 0.10066901  1.19950826], ……]
print('y_valid:'); print(y_valid[:5]) # [[ 1.], ……  [-1.]]

### ●ディープニューラルネットワークのモデル設計
- 入力の数（`INPUT_FEATURES`）は、$X_1$と$X_2$で**2つ**
- 隠れ層のレイヤー数は、**2つ**
  - 隠れ層にある1つ目のニューロンの数（`LAYER1_NEURONS`）は、**3つ**
  - 隠れ層にある2つ目のニューロンの数（`LAYER2_NEURONS`）は、**3つ**
- 出力層にあるニューロンの数（`OUTPUT_RESULTS`）は、**1つ**

#### リスト0-2　インポートと変数の宣言（全ての書き方に共通）

In [0]:
# ライブラリ「TensorFlow」のtensorflowパッケージを「tf」という別名でインポート
import tensorflow as tf
from tensorflow.keras import layers       # 「レイヤーズ」モジュールのインポート

# 定数（モデル定義時に必要となる数値）
INPUT_FEATURES = 2  # 入力（特徴）の数： 2
LAYER1_NEURONS = 3  # ニューロンの数： 3
LAYER2_NEURONS = 3  # ニューロンの数： 3
OUTPUT_RESULTS = 1  # 出力結果の数： 1

#### リスト0-3　正解率（精度）のカスタム指標（全ての書き方に共通）

In [0]:
import tensorflow.keras.backend as K

def tanh_accuracy(y_true, y_pred):           # y_trueは正解、y_predは予測（出力）
  threshold = K.cast(0.0, y_pred.dtype)              # -1か1かを分ける閾値を作成
  y_pred = K.cast(y_pred >= threshold, y_pred.dtype) # 閾値未満で0、以上で1に変換
  # 2倍して-1.0することで、0／1を-1.0／1.0にスケール変換して正解率を計算
  return K.mean(K.equal(y_true, y_pred * 2 - 1.0), axis=-1)

「[第5回　お勧めの、TensorFlow 2.0最新の書き方入門（エキスパート向け）](#scrollTo=PBSGSzs-RWFu)」の（4）以降に進む場合は、左記のリンクをクリックしてください。


## ■（1）Sequentialクラスのコンストラクター利用［tf.keras - Sequential API］

### ●モデル設計の書き方（初中級向け）

#### リスト1-1　Sequentialクラスのコンストラクターを利用した書き方

In [0]:
model = tf.keras.models.Sequential([    # モデルの生成

  # 隠れ層：1つ目のレイヤー
  layers.Dense(                 # 全結合層
      input_shape=(INPUT_FEATURES,),       # 入力の形状（＝入力層）
      name='layer1',                       # 表示用に名前付け
      kernel_initializer='glorot_uniform', # 重さの初期化（一様分布のランダム値）
      bias_initializer='zeros',            # バイアスの初期化（0）
      units=LAYER1_NEURONS,                # ユニットの数      
      activation='tanh'),                  # 活性化関数

  # 隠れ層：2つ目のレイヤー
  layers.Dense(                 # 全結合層
      name='layer2',                       # 表示用に名前付け
      kernel_initializer='glorot_uniform', # 重さの初期化
      bias_initializer='zeros',            # バイアスの初期化
      units=LAYER2_NEURONS,                # ユニットの数
      activation='tanh'),                  # 活性化関数

  # 出力層
  layers.Dense(                 # 全結合層
      name='layer_out',                    # 表示用に名前付け
      kernel_initializer='glorot_uniform', # 重さの初期化
      bias_initializer='zeros',            # バイアスの初期化
      units=OUTPUT_RESULTS,                # ユニットの数
      activation='tanh'),                  # 活性化関数

], name='sequential_constructor'           # モデルにも名前付け
)

# 以上でモデル設計は完了
model.summary()                            # モデルの内容を出力

#### リスト1-2　モデル構成図を表示するコード

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

### ●重み／バイアスの初期化指定【応用】

#### リスト1-3　重み／バイアスを初期化するための独自イニシャライザー

In [0]:
random_init = tf.keras.initializers.RandomUniform(
    minval=-1.0, maxval=1.0) # 下限と上限を指定した一様分布でランダム値

### ●学習と推論： 書き方入門（初中級者向け）で共通

#### リスト1-4　学習方法を設定、学習、推論（予測）するコード（共通）

In [0]:
# 学習方法を設定し、学習し、推論（予測）する
model.compile(tf.keras.optimizers.SGD(learning_rate=0.03), 'mean_squared_error', [tanh_accuracy])
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=15, epochs=100, verbose=1)
model.predict([[0.1,-0.2]])

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
train_loss = hist.history['loss']
valid_loss = hist.history['val_loss']
epochs = len(train_loss)
plt.plot(range(epochs), train_loss, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_loss, marker='.', label='loss (validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

モデルの書き方だけでなく、学習に関するコードも、下記のように2種類ある。

- **初中級者向けの書き方：** リスト1-4のように`compile()`＆`fit()`メソッドを呼び出す、簡単で手軽な書き方
- **エキスパート向けの書き方：** 次回説明するが、`tf.GradientTape`クラス（自動微分の記録機能）を使った柔軟で拡張性の高い書き方（PyTorchの書き方に近い）

### ●作成済みモデルのリセット

#### リスト1-5　次に進む前にモデルをリセットするコード

In [0]:
tf.keras.backend.clear_session() # 計算グラフを破棄する
del model                        # 変数を削除する

## ■（2）Sequentialオブジェクトのaddメソッドで追加［tf.keras - Sequential API］

### ●モデル設計の書き方（初中級向け）

#### リスト2-1　Sequentialオブジェクトのaddメソッドを利用した書き方

In [0]:
model = tf.keras.models.Sequential(  # モデルの生成
    name='sequential_add_method'     # モデルにも名前付け
)

# 隠れ層：1つ目のレイヤー
model.add(layers.Dense(        # 全結合層
    input_shape=(INPUT_FEATURES,),   # 入力の形状（＝入力層）
    name='layer1',                   # 表示用に名前付け
    units=LAYER1_NEURONS,            # ユニットの数
    activation='tanh'))              # 活性化関数

# 隠れ層：2つ目のレイヤー
model.add(layers.Dense(        # 全結合層
    name='layer2',                   # 表示用に名前付け
    units=LAYER2_NEURONS,            # ユニットの数
    activation='tanh'))              # 活性化関数

# 出力層
model.add(layers.Dense(        # 全結合層
    name='layer_out',                # 表示用に名前付け
    units=OUTPUT_RESULTS,            # ユニットの数
    activation='tanh'))              # 活性化関数

# 以上でモデル設計は完了
model.summary()                      # モデルの内容を出力

In [0]:
# モデルの構成図を表示
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

In [0]:
# 学習方法を設定し、学習し、推論（予測）する
model.compile(tf.keras.optimizers.SGD(learning_rate=0.03), 'mean_squared_error', [tanh_accuracy])
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=15, epochs=100, verbose=1)
model.predict([[0.1,-0.2]])

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
train_loss = hist.history['loss']
valid_loss = hist.history['val_loss']
epochs = len(train_loss)
plt.plot(range(epochs), train_loss, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_loss, marker='.', label='loss (validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [0]:
# 次に進む前にモデルをリセット
tf.keras.backend.clear_session() # グラフを破棄する
del model                        # 変数を削除する

## ■（3）Modelクラスのコンストラクター利用［tf.keras - Functional API］

### ●モデル設計の書き方（初中級向け）

#### リスト3-1　Modelクラスのコンストラクターを利用した書き方

In [0]:
# ### 活性化関数を変数（ハイパーパラメーター）として定義 ###
# 変数（モデル定義時に必要となる数値）
activation1 = layers.Activation('tanh' # 活性化関数（隠れ層用）： tanh関数（変更可能）
    , name='activation1'               # 活性化関数にも名前付け
    )
activation2 = layers.Activation('tanh' # 活性化関数（隠れ層用）： tanh関数（変更可能）
    , name='activation2'               # 活性化関数にも名前付け
    )
acti_out = layers.Activation('tanh'    # 活性化関数（出力層用）： tanh関数（固定）
    , name='acti_out'                  # 活性化関数にも名前付け
    )

# ### レイヤーを定義 ###
# input_shape引数の代わりに、Inputクラスを使うことも可能
inputs = layers.Input(          # 入力層
    name='layer_in',                 # 表示用に名前付け
    shape=(INPUT_FEATURES,))         # 入力の形状

# 隠れ層：1つ目のレイヤー
layer1 = layers.Dense(          # 全結合層
    #input_shape=(INPUT_FEATURES,),  # ※入力層は定義済みなので不要
    name='layer1',                   # 表示用に名前付け
    units=LAYER1_NEURONS)            # ユニットの数

# 隠れ層：2つ目のレイヤー
layer2 = layers.Dense(          # 全結合層
    name='layer2',                   # 表示用に名前付け
    units=LAYER2_NEURONS)            # ユニットの数

# 出力層
layer_out = layers.Dense(       # 全結合層
    name='layer_out',                # 表示用に名前付け
    units=OUTPUT_RESULTS)            # ユニットの数

# ### フォワードパスを定義 ###
# 「出力＝活性化関数（第n層（入力））」の形式で記述
x1 = activation1(layer1(inputs))     # 活性化関数は変数として定義
x2 = activation2(layer2(x1))         # 同上
outputs = acti_out(layer_out(x2))    # ※活性化関数は「tanh」固定

# ### モデルの生成 ###
model = tf.keras.Model(inputs=inputs, outputs=outputs
    , name='model_constructor'       # モデルにも名前付け
)

# ### 以上でモデル設計は完了 ###
model.summary()                      # モデルの内容を出力

In [0]:
# モデルの構成図を表示
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

In [0]:
# 学習方法を設定し、学習し、推論（予測）する
model.compile(tf.keras.optimizers.SGD(learning_rate=0.03), 'mean_squared_error', [tanh_accuracy])
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=15, epochs=100, verbose=1)
model.predict([[0.1,-0.2]])

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
train_loss = hist.history['loss']
valid_loss = hist.history['val_loss']
epochs = len(train_loss)
plt.plot(range(epochs), train_loss, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_loss, marker='.', label='loss (validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [0]:
# 次に進む前にモデルをリセット
tf.keras.backend.clear_session() # グラフを破棄する
del model                        # 変数を削除する

# 第5回　お勧めの、TensorFlow 2.0最新の書き方入門（エキスパート向け）

以下のコードを実行するには、事前に「[■コードの書き方を実行するための準備](#scrollTo=vT2IIbxs1mPe)」の全てのコードを実行しておく必要があります。

## ■（4）Modelクラスのサブクラス化［tf.keras - Subclassing／Imperative API］

### ●モデル設計の書き方（エキスパート向け）

#### リスト4-1　Modelクラスのサブクラス化を利用した書き方

In [0]:
# ### 活性化関数を変数（ハイパーパラメーター）として定義 ###
# 変数（モデル定義時に必要となる数値）
activation1 = layers.Activation('tanh' # 活性化関数（隠れ層用）： tanh関数（変更可能）
    , name='activation1'               # 活性化関数にも名前付け
    )
activation2 = layers.Activation('tanh' # 活性化関数（隠れ層用）： tanh関数（変更可能）
    , name='activation2'               # 活性化関数にも名前付け
    )
acti_out = layers.Activation('tanh'    # 活性化関数（出力層用）： tanh関数（固定）
    , name='acti_out'                  # 活性化関数にも名前付け
    )

# tf.keras.Modelをサブクラス化してモデルを定義
class NeuralNetwork(tf.keras.Model):

  # ### レイヤーを定義 ###
  def __init__(self, *args, **kwargs):
    super(NeuralNetwork, self).__init__(*args, **kwargs)

    # 入力層は定義「不要」。実際の入力によって決まるので

    # 隠れ層：1つ目のレイヤー
    self.layer1 = layers.Dense(          # 全結合層
        #input_shape=(INPUT_FEATURES,),  # 入力層（定義不要）
        name='layer1',                   # 表示用に名前付け
        units=LAYER1_NEURONS)            # ユニットの数

    # 隠れ層：2つ目のレイヤー
    self.layer2 = layers.Dense(          # 全結合層
        name='layer2',                   # 表示用に名前付け
        units=LAYER2_NEURONS)            # ユニットの数

    # 出力層
    self.layer_out = layers.Dense(       # 全結合層
        name='layer_out',                # 表示用に名前付け
        units=OUTPUT_RESULTS)            # ユニットの数

  # ### フォワードパスを定義 ###
  def call(self, inputs, training=None): # 入力と、訓練／評価モード
    # 「出力＝活性化関数（第n層（入力））」の形式で記述
    x1 = activation1(self.layer1(inputs))     # 活性化関数は変数として定義
    x2 = activation2(self.layer2(x1))         # 同上
    outputs = acti_out(self.layer_out(x2))    # ※活性化関数は「tanh」固定
    return outputs

# ### モデルの生成 ###
# model = tf.keras.Model(inputs=inputs, outputs=outputs
#     , name='model_constructor'       # モデルにも名前付け
# )
model = NeuralNetwork()                # モデルの生成

# ### 以上でモデル設計は完了 ###
#model.summary()  # モデル内容の出力はできない！（後述）

### ●モデルの内容と構成図の出力について

#### リスト4-2　計算グラフなしでは「model.summary()」は失敗する

In [0]:
# モデルの内容を出力
#model.summary()     # エラー！ ← 計算グラフが構築できていないため

# エラー出力例
# ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

「エラー出力例」の日本語翻訳： 「ValueError: このモデルはまだビルド（＝計算グラフが構築）されていません。`build()`を呼び出すか、データを指定して`fit()`を呼び出すか、自動ビルドのために最初のレイヤーで`input_shape`引数を指定するかして、先にモデルをビルドしてください。」

### ○方法1

#### リスト4-3　方法1：推論して動的に計算グラフを構築する

In [0]:
model1 = NeuralNetwork(name='subclassing_model1') # モデルの生成

# 方法1： 推論して動的にグラフを構築する
temp_input = [[0.1,-0.2]]                         # 仮の入力値
temp_output = model1.predict(temp_input)          # 推論の実行

# モデルの内容を出力
model1.summary()

# モデルの構成図を表示
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

### ○方法2

#### リスト4-4　方法2：入力形状を指定して計算グラフを構築する（モデルのビルド）

In [0]:
model2 = NeuralNetwork(name='subclassing_model2') # モデルの生成

# 方法2： 入力形状を指定して計算グラフを構築する
model2.build(                # モデルのビルド
   input_shape=(None,2))     # 入力の形状（＝入力層）※タプル形式

# モデルの内容を出力
model2.summary()

# モデルの構成図を表示
tf.keras.utils.plot_model(model2, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

### ○方法3

#### リスト4-5　方法3：モデル構造の出力用に「仮のモデル」をFunctional APIで生成する（裏技テクニック）

In [0]:
model3 = NeuralNetwork()                # モデルの生成

# 「仮のモデル」をFunctional APIで生成する独自関数
def get_functional_model(model):
  # このコードは、「リスト3-1のFunctional API」とほぼ同じ
  x = layers.Input(shape=(INPUT_FEATURES,), name='layer_in')
  temp_model = tf.keras.Model(
      inputs=[x],
      outputs=model.call(x),  # ※サブクラス化したモデルの`call`メソッドを指定
      name='subclassing_model3')  # 仮モデルにも名前付け
  return temp_model

# Functional APIの「仮のモデル」を取得
f_model = get_functional_model(model3)

# モデルの内容を出力
f_model.summary()

# モデルの構成図を表示
tf.keras.utils.plot_model(f_model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

### ●学習と推論： 書き方入門（エキスパート向け）

【念のため、再説明】モデルの書き方だけでなく、学習に関するコードも、下記のように2種類ある。

- **初中級者向けの書き方：** `compile()`＆`fit()`メソッドを呼び出す、簡単で手軽な書き方
- **エキスパート向けの書き方：** `tf.GradientTape`クラス（自動微分の記録機能）を使った柔軟で拡張性の高い書き方（PyTorchの書き方に近い）

### ○初中級者向け書き方の復習

#### リスト4-6　学習方法を設定、学習、推論（予測）するコード（共通）の振り返り

「リスト1-4　学習方法を設定、学習、推論（予測）するコード（共通）」と同じコードの再掲。


In [0]:
# 学習方法を設定し、学習し、推論（予測）する
model.compile(tf.keras.optimizers.SGD(learning_rate=0.03), 'mean_squared_error', [tanh_accuracy])
hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=15, epochs=100, verbose=1)
model.predict([[0.1,-0.2]])

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
train_loss = hist.history['loss']
valid_loss = hist.history['val_loss']
epochs = len(train_loss)
plt.plot(range(epochs), train_loss, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_loss, marker='.', label='loss (validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

### リスト4-x　次に進む前にモデルをリセットするコード

「リスト1-5　次に進む前にモデルをリセットするコード」と同じコードの再掲（※同じモデルを使うので再度インスタンス化するコードも追加した）。

In [0]:
# 次のコードのためにモデル（重みやバイアス）をリセットする
tf.keras.backend.clear_session() # 計算グラフを破棄する
del model                        # 変数を削除する

model = NeuralNetwork()          # モデルの再生成

#### リスト4-x　評価関数のカスタム実装

In [0]:
import tensorflow.keras.backend as K

# カスタムの評価関数を実装（TensorFlow低水準API利用）
# （tf.keras.metrics.binary_accuracy()の代わり）
def tanh_accuracy(y_true, y_pred):           # y_trueは正解、y_predは予測（出力）
  threshold = K.cast(0.0, y_pred.dtype)              # -1か1かを分ける閾値を作成
  y_pred = K.cast(y_pred >= threshold, y_pred.dtype) # 閾値未満で0、以上で1に変換
  # 2倍して-1.0することで、0／1を-1.0／1.0にスケール変換して正解率を計算
  return K.mean(K.equal(y_true, y_pred * 2 - 1.0), axis=-1)

# カスタムの評価関数クラスを実装（サブクラス化）
# （tf.keras.metrics.BinaryAccuracy()の代わり）
class TanhAccuracy(tf.keras.metrics.Mean):
  def __init__(self, name='tanh_accuracy', dtype=None):
    super(TanhAccuracy, self).__init__(name, dtype)

  # 正解率の状態を更新する際に呼び出される関数をカスタマイズ
  def update_state(self, y_true, y_pred, sample_weight=None):
    matches = tanh_accuracy(y_true, y_pred)
    return super(TanhAccuracy, self).update_state(
        matches, sample_weight=sample_weight)

### ○推論

#### リスト4-7　推論（エキスパート向け）

In [0]:
# 【初中級者向けとの比較用】推論する
# python_input2 = [[0.1,-0.2]]
# predict2 = model.predict(python_input2)
# print(predict2)
# # [[0.9845763]] ……などと表示される

# ###【エキスパート向け】推論する ###
python_input1 = [[0.1,-0.2]]   # 入力値（Pythonリスト値／NumPy多次元配列値）

# Pythonリスト値やNumPy多次元配列値はテンソルにいったん変換する必要がある
tensor_input1 = tf.convert_to_tensor(python_input1, dtype=tf.float32)
#tensor_input1 = tf.constant([[0.1,-0.2]])  # 入力値（定数）
#tensor_input1 = tf.Variable([[0.1,-0.2]])  # 入力値（変数）

predict1 = model(tensor_input1)         # ※テンソルの入力しか受け付けない

print(predict1)                         # ※テンソルが出力される
# tf.Tensor([[0.9845763]], shape=(1, 1), dtype=float32) ……などと表示される

### ○オプティマイザ（最適化用オブジェクト）

#### リスト4-8-1　オプティマイザ（最適化用オブジェクト）の定義（エキスパート向け）

In [0]:
# 【初中級者向けとの比較用】学習方法を設定する
# model.compile(
#     tf.keras.optimizers.SGD(learning_rate=0.03),        # 最適化アルゴリズム
#     ……損失関数……,
#     ……評価関数……)

# ###【エキスパート向け】最適化アルゴリズムを定義する ###
optimizer = tf.keras.optimizers.SGD(learning_rate=0.03)  # 更新時の学習率

### ○損失関数

#### リスト4-8-2　損失関数の定義（エキスパート向け）

In [0]:
# 【初中級者向けとの比較用】学習方法を設定する
# model.compile(
#     ……最適化アルゴリズム……,
#     'mean_squared_error',  # 損失関数
#     ……評価関数……)

# ###【エキスパート向け】損失関数を定義する ###
criterion = tf.keras.losses.MeanSquaredError()

### ○評価関数

#### リスト4-8-3　評価関数の定義（エキスパート向け）

In [0]:
# 【初中級者向けとの比較用】学習方法を設定する
# model.compile(
#     ……最適化アルゴリズム……,
#     ……損失関数……,
#     [tanh_accuracy])   # 評価関数

# ### 【エキスパート向け】評価関数を定義する ###
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = TanhAccuracy(name='train_accuracy')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = TanhAccuracy(name='valid_accuracy')

### ○tf.dataデータセット

#### リスト4-9-1　ミニバッチ用データセットの準備（エキスパート向け）

In [0]:
# 【初中級者向けとの比較用】入力データを指定して学習する
# model.fit(
#     X_train, y_train,                    # 訓練データ
#     validation_data=(X_valid, y_valid),  # 精度検証データ
#     batch_size=15,                       # バッチサイズ
#     ……エポック数……,
#     ……実行状況の出力モード……)

# ###【エキスパート向け】入力データを準備する ###

# NumPy多次元配列のデータ型をデータセット（テンソル）用に統一する
X_train_f32 = X_train.astype('float32') # np.float64型 → np.float32型
y_train_f32 = y_train.astype('float32') # 同上
X_valid_f32 = X_valid.astype('float32') # 同上
y_valid_f32 = y_valid.astype('float32') # 同上

# 「入力データ（X）」と「教師ラベル（y）」を、1つの「スライスデータセット（TensorSliceDataset）」にまとめる
train_sliced = tf.data.Dataset.from_tensor_slices((X_train_f32, y_train_f32)) # 訓練用
valid_sliced = tf.data.Dataset.from_tensor_slices((X_valid_f32, y_valid_f32)) # 精度検証用

# シャッフルして（訓練データのみ）、ミニバッチ用の「バッチデータセット（BatchDataset）」にする
train_dataset = train_sliced.shuffle(250).batch(15)
valid_dataset = valid_sliced.batch(15)
print(train_dataset)
# <BatchDataset shapes: ((None, 2), (None, 1)), types: (tf.float32, tf.float32)> ……などと表示される

### ○学習（1回分）

#### リスト4-9-2　1回分の「訓練」と「精度検証」の処理（エキスパート向け）

In [0]:
import tensorflow.keras.backend as K

# ###【エキスパート向け】訓練する（1回分） ###
@tf.function
def train_step(train_X, train_y):
  # 訓練モードに設定
  training = True
  K.set_learning_phase(training)  # tf.keras内部にも伝える

  with tf.GradientTape() as tape: # 勾配をテープに記録
    # フォワードプロパゲーションで出力結果を取得
    #train_X                                   # 入力データ
    pred_y = model(train_X, training=training) # 出力結果
    #train_y                                   # 正解ラベル

    # 出力結果と正解ラベルから損失を計算し、勾配を求める
    loss = criterion(pred_y, train_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得
  
  # 逆伝播の処理として勾配を計算（自動微分）
  gradient = tape.gradient(loss, model.trainable_weights)

  # 勾配を使ってパラメーター（重みとバイアス）を更新
  optimizer.apply_gradients(zip(gradient, model.trainable_weights)) # 指定されたデータ分の最適化を実施

  # 損失と正解率を算出して保存
  train_loss(loss)
  train_accuracy(train_y, pred_y)

# ###【エキスパート向け】精度検証する（1回分） ###
@tf.function
def valid_step(valid_X, valid_y):
  # 評価モードに設定（※dropoutなどの挙動が評価用になる）
  training = False
  K.set_learning_phase(training)  # tf.keras内部にも伝える

  # フォワードプロパゲーションで出力結果を取得
  #valid_X                                   # 入力データ
  pred_y = model(valid_X, training=training) # 出力結果
  #valid_y                                   # 正解ラベル

  # 出力結果と正解ラベルから損失を計算
  loss = criterion(pred_y, valid_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得
  # ※評価時は勾配を計算しない

  # 損失と正解率を算出して保存
  valid_loss(loss)
  valid_accuracy(valid_y, pred_y)


### ○学習（ループ処理）

#### リスト4-10　「訓練」と「精度検証」をバッチサイズ単位でエポック回繰り返す（エキスパート向け）

- 1つ目のforループでエポックを回している
  - 2つ目のforループでバッチ単位分のデータを処理に渡している（ミニバッチ処理）
    - `train_step`関数で訓練を、`valid_step`関数で評価を1回だけ実行する。


In [0]:
# 【初中級者向けとの比較用】入力データを指定して学習する
# model.fit(
#     ……訓練データ（入力）……, ……同（ラベル）……,
#     ……精度検証データ……,
#     ……バッチサイズ……,
#     epochs=100,  # エポック数
#     verbose=1)   # 実行状況の出力モード

# ###【エキスパート向け】学習する ###

# 定数（学習／評価時に必要となるもの）
EPOCHS = 100             # エポック数： 100

# 損失の履歴を保存するための変数
train_history = []
valid_history = []

for epoch in range(EPOCHS):
  # エポックのたびに、メトリクスの値をリセット
  train_loss.reset_states()      # 「訓練」時における累計「損失値」
  train_accuracy.reset_states()  # 「訓練」時における累計「正解率」
  valid_loss.reset_states()      # 「評価」時における累計「損失値」
  valid_accuracy.reset_states()  # 「評価」時における累計「正解率」

  for train_X, train_y in train_dataset:
    # 【重要】1ミニバッチ分の「訓練（学習）」を実行
    train_step(train_X, train_y)
          
  for valid_X, valid_y in valid_dataset:
    # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
    valid_step(valid_X, valid_y)

  # ミニバッチ単位で累計してきた損失値や正解率の平均を取る
  n = epoch + 1                          # 処理済みのエポック数
  avg_loss = train_loss.result()         # 訓練用の平均損失値
  avg_acc = train_accuracy.result()      # 訓練用の平均正解率
  avg_val_loss = valid_loss.result()     # 訓練用の平均損失値
  avg_val_acc = valid_accuracy.result()  # 訓練用の平均正解率

  # グラフ描画のために損失の履歴を保存する
  train_history.append(avg_loss)
  valid_history.append(avg_val_loss)

  # 損失や正解率などの情報を表示
  print(f'[Epoch {n:3d}/{EPOCHS:3d}]' \
        f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
        f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')

print('Finished Training')
print(model.get_weights())  # 学習後のパラメーターの情報を表示

### ○評価

#### リスト4-11　損失値の推移グラフ描画

In [0]:
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [0]:
# 次に進む前にモデルをリセット
tf.keras.backend.clear_session() # グラフを破棄する
del model                        # 変数を削除する

（つづく。以下に対応する記事は未公開です。数週間のうちに公開予定です。）

# 第6回　カスタマイズするための、TensorFlow 2.0最新の書き方入門（エキスパート向け）

## ■（5）TensorFlow低水準APIでカスタム実装［TensorFlow Low-level API］

### ●活性化関数のカスタム実装

活性化関数については、Sequentialモデル（リスト1-1／2-1）では`'tanh'`という文字列を、Functional API（リスト3-1）やSubclassingモデル（リスト4-1）では`tf.keras.layers.Activation`クラスを使用したが、`tf.keras.activations.tanh()`関数や`tf.nn.tanh()`関数を使用することも可能である。TensorFlow 2.xで利用可能な活性化関数（文字列／関数）には主に以下のものがある。
- `'sigmoid'`： `tf.keras.activations.sigmoid()`関数
- `'hard_sigmoid'`： `tf.keras.activations.hard_sigmoid()`関数
- `'tanh'`： `tf.keras.activations.tanh()`関数
- `'relu',`： `tf.keras.activations.relu()`関数
- `'elu'`： `tf.keras.activations.elu()`関数
- `'selu'`： `tf.keras.activations.selu()`関数
- `'swish'`： `tf.keras.activations.swish()`関数
- `'softsign'`： `tf.keras.activations.softsign()`関数
- `'softplus'`： `tf.keras.activations.softplus()`関数
- `'softmax'`： `tf.keras.activations.softmax()`関数
- `'linear'`： `tf.keras.activations.linear()`関数
- `'exponential'`： `tf.keras.activations.exponential()`関数
- `tf.nn.sigmoid()`関数
- `tf.nn.tanh()`関数
- `tf.nn.relu()`関数
- `tf.nn.relu6()`関数
- `tf.nn.leaky_relu()`関数
- `tf.nn.elu()`関数
- `tf.nn.selu()`関数
- `tf.nn.swish()`関数
- `tf.nn.softsign()`関数
- `tf.nn.softplus()`関数
- `tf.nn.softmax()`関数

詳しくは下記のリンク先を参照してほしい。
- 詳細：[Module: tf.keras.activations ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
- 詳細：[Module: tf.nn  |  TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/nn)
- 詳細：[Module: tf.keras.layers.Activation ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Activation)

目的の活性化関数が見つからない場合は、次のようにしてカスタム（独自）のものを作成することが可能だ。

In [0]:
# カスタムの活性化関数のPython関数を実装
def custom_activation(x):
    return (tf.exp(x)-tf.exp(-x))/(tf.exp(x)+tf.exp(-x))

# # TensorFlow低水準APIではなく、Keras backend APIを使ってもOK
# import tensorflow.keras.backend as K
# # 上記のインポートを行ったうえで、以下の内容で上記のコードを書き換えるだけ
# # 「tf.exp」 → 「K.exp」

# カスタムの活性化関数クラスを実装（レイヤーのサブクラス化）
# （tf.keras.layers.Activation()の代わり）
class CustomActivation(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super(CustomActivation, self).__init__(**kwargs)

  def call(self, inputs):
    return custom_activation(inputs)

### ●レイヤーのカスタム実装

レイヤーについては、Sequentialモデル／Functional API／Subclassingモデル（リスト1-1／2-1／3-1／4-1）いずれも`tf.keras.layers.Dense`クラスを使用したが、それ以外にも多種多様なものが用意されている。TensorFlow 2.xで利用可能な代表的なレイヤーには以下のものがある。
- `Dense`クラス
- `Activation`クラス
- `Dropout`クラス
  - `SpatialDropout1D`クラス
  - `SpatialDropout2D`クラス
  - `SpatialDropout3D`クラス
- `Flatten`クラス
- `InputLayer`クラス
- `Reshape`クラス
- `Permute`クラス
- `RepeatVector`クラス
- `Lambda`クラス
- `ActivityRegularization`クラス
- `Masking`クラス

詳しくは下記のリンク先を参照してほしい。
- 詳細：[Module: tf.keras.layers ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/layers)

目的のレイヤーが見つからない場合は、次のようにしてカスタム（独自）のものを作成することが可能だ。

In [0]:
# カスタムの全結合層（線形変換）のPython関数を実装
def fully_connected(inputs, weights, bias):
  return tf.matmul(inputs, weights) + bias

# # TensorFlow低水準APIではなく、Keras backend APIを使ってもOK
# import tensorflow.keras.backend as K
# # 上記のインポートを行ったうえで、以下の内容で上記のコードを書き換えるだけ
# 「tf.matmul」 → 「K.dot」

# カスタムのレイヤークラスを実装（レイヤーのサブクラス化）
# （tf.keras.layers.Dense()の代わり）
class CustomLayer(tf.keras.layers.Layer):
  def __init__(self, units, input_dim=None, **kwargs):
    self.input_dim = input_dim  # 入力の次元数（＝レイヤーの入力数）
    self.units = units          # ニューロン数（＝レイヤーの出力数）
    super(CustomLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    #print(input_shape) # 入力形状。例えば「(2, 2)」＝2行2列なら入力の次元数は2列
    input_data_dim = input_shape[-1] # 入力の次元数（＝レイヤーの入力数）

    # 入力の次元数をチェック（デバッグ）
    if self.input_dim != None:
      assert input_data_dim == self.input_dim  # 指定された入力次元数と実際の入力次元数が異なります

    # 重みにを追加する
    self.kernel = self.add_weight(
        shape=(input_data_dim, self.units),
        name='kernel',
        initializer='glorot_uniform',  # リスト1-3のような独自の関数も指定できる
        trainable=True)

    # バイアスを追加する
    self.bias = self.add_weight(
        shape=(self.units,),
        name='bias',
        initializer='zeros',
        trainable=True)
    
    #self.built = True # Layerクラスでビルド済みどうかを管理するのに使われている（なくても大きな問題はない）
    super(CustomLayer, self).build(input_shape) # 上と同じ意味。APIドキュメントで推奨されている書き方

  def call(self, inputs):
    return fully_connected(inputs, self.kernel, self.bias)

  # # for Keras to do automatic shape inference.
  # def compute_output_shape(self, input_shape): # 純正Kerasは実装をするとよみみたいだが、tf.kerasでは呼ばれないようなので実装しなくてよい. tensorflow側のチュートリアルでは記載されていない
  #   # [The inheriting keras.layers.Layer does not call a compute_output_shape after switching to tf.keras from keras · Issue #33785 · tensorflow/tensorflow](https://github.com/tensorflow/tensorflow/issues/33785)
  #   #   print(input_shape)
  #   return (input_shape[0], self.units) # （データの行数=None、出力ユニット数）一応実装
  #   # 作成したレイヤーの内部で入力から出力でshapeを変更する場合には，ここでshape変換のロジックを指定する必要があります．こうすることでKerasが自動的にshapeを推定します．
  # tf.kerasでは理由は明確には分からないが実装すべきメソッドとしては挙げられていない（compute_output_shape）

### ●オプティマイザ（最適化アルゴリズム）のカスタム実装

- `tf.optimizers`はエイリアスで`tf.keras.optimizers`を指す

オプティマイザ（最適化アルゴリズム）については、Sequentialモデル／Functional API／Subclassingモデル（リスト1-1／2-1／3-1／4-1）いずれも`tf.keras.optimizers.SGD`クラスを使用したが、それ以外にも用意されている。TensorFlow 2.xで利用可能な代表的なオプティマイザには以下のものがある。
- `SGD`クラス（確率的勾配降下法）
- `RMSprop`クラス
- `Adagrad`クラス
- `Adadelta`クラス
- `Adam`クラス
- `Adamax`クラス
- `Nadam`クラス
- `Ftrl`クラス

詳しくは下記のリンク先を参照してほしい。
- [Module: tf.keras.optimizers ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)

目的のオプティマイザが見つからない場合は、次のようにしてカスタム（独自）のものを作成することが可能だ。ただし簡単ではないので、説明は割愛する。基本的には、「[tensorflow/gradient_descent.py at master · tensorflow/tensorflow](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/optimizer_v2/gradient_descent.py)」をまねればよい。以下のコードも、SGDのコードを参考にして作ったものである。

In [0]:
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.ops import resource_variable_ops
from tensorflow.python.training import training_ops

# カスタムの最適化アルゴリズムクラスを実装（オプティマイザのサブクラス化）
# （tf.keras.optimizers.SGDの代わり）
class CustomOptimizer(optimizer_v2.OptimizerV2):
  def __init__(self, learning_rate=0.01, name="CustomOptimizer", **kwargs):
    super(CustomOptimizer, self).__init__(name, **kwargs)
    self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))

  def _create_slots(self, var_list):
    pass

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(CustomOptimizer, self)._prepare_local(var_device, var_dtype, apply_state)

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))
    return training_ops.resource_apply_gradient_descent(
        var.handle, coefficients["lr_t"], grad, use_locking=self._use_locking)

  def _resource_apply_sparse_duplicate_indices(self, grad, var, indices, **kwargs):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = (kwargs.get("apply_state", {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))
    return resource_variable_ops.resource_scatter_add(
        var.handle, indices, -grad * coefficients["lr_t"])

  def get_config(self):
    config = super(CustomOptimizer, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate")
    })
    return config

### ●損失関数の定義のカスタム実装

- `tf.losses`はエイリアスで`tf.keras.losses`を指す

損失関数については、初中級者向け（リスト1-1／2-1／3-1）では`'mean_squared_error'`という文字列を、エキスパート向け（リスト4-1）では`tf.keras.losses.MeanSquaredError`クラスを使用したが、`mean_squared_error()`関数を使用することも可能である。TensorFlow 2.xで利用可能な損失関数（文字列／関数／クラス）には主に以下のものがある。
- `MeanSquaredError`クラス（略記：`MSE`）： `mean_squared_error()`関数
- `MeanAbsoluteError`クラス（略記：`MAE`）： `mean_absolute_error()`関数
- `MeanAbsolutePercentageError`クラス（略記：`MAPE`）： `mean_absolute_percentage_error()`関数
- `MeanSquaredLogarithmicError`クラス（略記：`MSLE`）： `mean_squared_logarithmic_error()`関数
- `SquaredHinge`クラス： `squared_hinge()`関数
- `Hinge`クラス： `hinge()`関数
- `CategoricalHinge`クラス： `categorical_hinge()`関数
- `LogCosh`クラス： `logcosh()`関数
- `Huber`クラス： `huber_loss()`関数
- `CategoricalCrossentropy`クラス： `categorical_crossentropy()`関数
- `SparseCategoricalCrossentropy`クラス： `sparse_categorical_crossentropy()`関数
- `BinaryCrossentropy`クラス： `binary_crossentropy()`関数
- `KLDivergence`クラス（略記：`KLD`）： `kullback_leibler_divergence()`関数
- `Poisson`クラス： `poisson()`関数
- `CosineSimilarity`クラス： `cosine_proximity()`関数

詳しくは下記のリンク先を参照してほしい。
- 詳細： [Module: tf.keras.losses ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/losses)

目的の損失関数が見つからない場合は、次のようにしてカスタム（独自）のものを作成することが可能だ。


In [0]:
# カスタムの損失関数のPython関数を実装
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# # TensorFlow低水準APIではなく、Keras backend APIを使ってもOK
# import tensorflow.keras.backend as K
# # 上記のインポートを行ったうえで、以下の内容で上記のコードを書き換えるだけ
# # 「tf.reduce_mean」 → 「K.mean」
# # 「tf.square」 → 「K.square」

# カスタムの損失関数クラスを実装（レイヤーのサブクラス化）
# （tf.keras.losses.MeanSquaredError()の代わり）
class Customloss(tf.keras.losses.Loss):
  def __init__(self, name="custom_loss", **kwargs):
    super(Customloss, self).__init__(name=name, **kwargs)

  def call(self, y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)  # 念のためTensor化
    y_true = tf.cast(y_true, y_pred.dtype) # 念のため同じデータ型化
    return custom_loss(y_true, y_pred)

### ●評価関数（正解率や損失）のカスタム実装

- `tf.metrics`はエイリアスで`tf.keras.metrics`を指す

評価関数については、初中級者向け（リスト1-1／2-1／3-1）では独自の`tanh_accuracy`関数を、エキスパート向け（リスト4-1）では`tf.keras.metrics.Mean`クラスなどを使用したが、`'binary_accuracy'`文字列や`binary_accuracy()`関数を使用することも可能である。TensorFlow 2.xで利用可能な評価関数（文字列／関数／クラス）には主に以下のものがある。
- `Accuracy`クラス： `accuracy()`関数
- `BinaryAccuracy`クラス： `binary_accuracy()`関数
- `CategoricalAccuracy`クラス： `categorical_accuracy()`関数
- `SparseCategoricalAccuracy`クラス： `sparse_categorical_accuracy()`関数
- `TopKCategoricalAccuracy`クラス： `top_k_categorical_accuracy()`関数
- `SparseTopKCategoricalAccuracy`クラス： `sparse_top_k_categorical_accuracy()`関数
- `FalsePositives`クラス
- `FalseNegatives`クラス
- `TrueNegatives`クラス
- `TruePositives`クラス
- `Precision`クラス
- `Recall`クラス
- `SensitivityAtSpecificity`クラス
- `SpecificityAtSensitivity`クラス
- `PrecisionAtRecall`クラス
- `AUC`クラス

詳しくは下記のリンク先を参照してほしい。
- 詳細：[Module: tf.keras.metrics ｜ TensorFlow Core v2.x](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

目的の評価関数が見つからない場合は、次のようにしてカスタム（独自）のものを作成することが可能だ。


In [0]:
# 正解かどうかを判定する関数を実装
def custom_matches(y_true, y_pred):         # y_trueは正解、y_predは予測（出力）
  threshold = tf.cast(0.0, y_pred.dtype)              # -1か1かを分ける閾値を作成
  y_pred = tf.cast(y_pred >= threshold, y_pred.dtype) # 閾値未満で0、以上で1に変換
  # 2倍して-1.0することで、0／1を-1.0／1.0にスケール変換して正解率を計算
  return tf.equal(y_true, y_pred * 2 - 1.0) # 正解かどうかのデータ（平均はしていない）

# # TensorFlow低水準APIではなく、Keras backend APIを使ってもOK
# import tensorflow.keras.backend as K
# # 上記のインポートを行ったうえで、以下の内容で上記のコードを書き換えるだけ
# 「tf.cast」 → 「K.cast」
# 「tf.equal」 → 「K.equal」

# カスタムの評価関数クラスを実装（サブクラス化）
# （tf.keras.metrics.BinaryAccuracy()の代わり）
class CustomAccuracy(tf.keras.metrics.Mean):
  def __init__(self, name='custom_accuracy', dtype=None):
    super(CustomAccuracy, self).__init__(name, dtype)

  # 正解率の状態を更新する際に呼び出される関数をカスタマイズ
  def update_state(self, y_true, y_pred, sample_weight=None):
    matches = custom_matches(y_true, y_pred)
    return super(CustomAccuracy, self).update_state(
        matches, sample_weight=sample_weight) # ※平均は内部で自動的に取ってくれる

### ●ここまでのカスタム実装の使用例

In [0]:
# 活性化関数の定義
activation1 = CustomActivation(name='activation1')
activation2 = CustomActivation(name='activation2')
acti_out = layers.Activation('tanh', name='acti_out')

# モデルの設計
class NeuralNetwork(tf.keras.Model):
  def __init__(self, *args, **kwargs):
    super(NeuralNetwork, self).__init__(*args, **kwargs)
    self.layer1 = CustomLayer(units=LAYER1_NEURONS, name='layer1')
    self.layer2 = CustomLayer(units=LAYER2_NEURONS, name='layer2')
    self.layer_out = CustomLayer(units=OUTPUT_RESULTS, name='layer_out')
  def call(self, inputs, training=None):
    x1 = activation1(self.layer1(inputs))
    x2 = activation2(self.layer2(x1))
    outputs = acti_out(self.layer_out(x2))
    return outputs

# モデルの生成
model = NeuralNetwork()

# モデルの概要と構成図の出力
def get_functional_model(model):
  x = layers.Input(shape=(INPUT_FEATURES,), name='layer_in')
  temp_model = tf.keras.Model(inputs=[x], outputs=model.call(x), name='cusotm_model')
  return temp_model
f_model = get_functional_model(model)
f_model.summary()
tf.keras.utils.plot_model(f_model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=False, filename='model.png')

In [0]:
# 最適化アルゴリズムと損失関数の定義
optimizer = CustomOptimizer(learning_rate=0.03)
criterion = Customloss()

# 評価関数（損失と正解率）の定義
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = CustomAccuracy(name='train_accuracy')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = CustomAccuracy(name='valid_accuracy')

# ミニバッチ用データの準備
X_train_f32 = X_train.astype('float32')
y_train_f32 = y_train.astype('float32')
X_valid_f32 = X_valid.astype('float32')
y_valid_f32 = y_valid.astype('float32')
train_sliced = tf.data.Dataset.from_tensor_slices((X_train_f32, y_train_f32)) # 訓練用
valid_sliced = tf.data.Dataset.from_tensor_slices((X_valid_f32, y_valid_f32)) # 精度検証用
train_dataset = train_sliced.shuffle(60000).batch(15)
valid_dataset = valid_sliced.batch(15)

In [0]:
import tensorflow.keras.backend as K

# 1回分の訓練（学習）の処理
@tf.function
def train_step(train_X, train_y):
  training = True
  K.set_learning_phase(training)
  with tf.GradientTape() as tape:
    pred_y = model(train_X, training=training)
    loss = criterion(pred_y, train_y)
  gradient = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(gradient, model.trainable_weights))
  train_loss(loss)
  train_accuracy(train_y, pred_y)

# 1回分の精度検証（評価）の処理
@tf.function
def valid_step(valid_X, valid_y):
  training = False
  K.set_learning_phase(training)
  pred_y = model(valid_X, training=training)
  loss = criterion(pred_y, valid_y)
  valid_loss(loss)
  valid_accuracy(valid_y, pred_y)

# エポック数分、訓練（学習）と精度検証（評価）を実行
EPOCHS = 100
train_history = []
valid_history = []
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  valid_loss.reset_states()
  valid_accuracy.reset_states()
  for train_X, train_y in train_dataset:
    train_step(train_X, train_y)
  for valid_X, valid_y in valid_dataset:
    valid_step(valid_X, valid_y)   
  n = epoch + 1
  avg_loss = train_loss.result()
  avg_acc = train_accuracy.result()
  avg_val_loss = valid_loss.result()
  avg_val_acc = valid_accuracy.result()
  train_history.append(avg_loss)
  valid_history.append(avg_val_loss)
  print(f'[Epoch {epoch+1:3d}/{EPOCHS:3d}]' \
        f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
        f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
print('Finished Training')
print(model.get_weights())

In [0]:
# 学習結果（損失）のグラフを描画
import matplotlib.pyplot as plt
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [0]:
# 推論
model(tf.constant([[0.1,-0.2]]))
# <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.88659185]], dtype=float32)> ……などと表示される

In [0]:
# 次に進む前にモデルをリセット
tf.keras.backend.clear_session() # グラフを破棄する
del model                        # 変数を削除する

## ■（6）作成済みEstimatorsの利用［tf.estimator］（※互換性のためで非推奨）

「作成済みEstimators（Premade Estimators）」ファミリーの中では、以下はTensorFlow 2.0 APIでサポートされている。

- `tf.estimator.LinearClassifier`： 線形モデル（分類用）
- `tf.estimator.LinearRegressor`： 線形モデル（回帰用）
- `tf.estimator.DNNClassifier`：ニューラルネットワークモデル（分類用）
- `tf.estimator.DNNRegressor`：ニューラルネットワークモデル（回帰用）
- `tf.estimator.DNNLinearCombinedClassifier`：ニューラルネットワークと線形の結合モデル（分類）
- `tf.estimator.DNNLinearCombinedRegressor`：ニューラルネットワークと線形の結合モデル（回帰）

この中で有用なのは、`DNNClassifier`と`DNNRegressor`の2つである。

なお、結合モデルとは、線形モデル（Wideモデル）とニューラルネットワークモデル（Deepモデル）を結合させた**Wide＆Deepモデル**のことでである（詳しくは「[Google AI Blog: Wide & Deep Learning: Better Together with TensorFlow](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html)」を参照してほしい）。

tf.estimatorは、TensorFlow 2.x（2.0以降）でもサポートされているものの、TensorFlow 1.xからの移行用として残されている。将来的に廃止／削除される可能性は非常に高いと思われるので（※筆者の確認によると、実際に、コードが2019年8月以来、何も変更されずに放置されているようだった）、「基本的に使わない方がよい」だろう。公式ドキュメントでも、新規開発であればtf.keras（TensorFlow＋Keras）を使うことが推奨されている。

ここでは、あくまで「Estimatorのコードを見たときに戸惑わない」ように、知識としてコードを紹介するにとどめる。コードの書き方も、あまり洗練されておらず、面倒くさい（と筆者は感じた）。


### リスト6-1　データの準備（Pandasのデータフレーム）

In [0]:
# 列名を指定するため、ライブラリ「Pandas」のデータフレーム化
import pandas as pd
train_X = pd.DataFrame(X_train.astype('float32'), columns=['x', 'y'])
train_y = pd.DataFrame(y_train.astype('float32'))
valid_X = pd.DataFrame(X_valid.astype('float32'), columns=['x', 'y'])
valid_y = pd.DataFrame(y_valid.astype('float32'))

データは、訓練データ（`train`）と精度検証データ（`valid`＝評価）の2つに分割されており、それぞれ入力（＝特徴）データ（$X$：行列）とラベル（$y$：ベクトル）がある。これらを32bitの浮動小数点数値（float32）に変えた上で、データ解析支援ライブラリ「Pandas」のデータフレームにしている。`columns=['x', 'y']`引数は、列名として「x」と「y」（X座標とY座標）を指定している。わざわざPandasを使っている理由は、この列名指定が簡単であるのと、`tf.data`がPandasに対応しているからである。ちなみに`tf.data`は、前掲の「TensorFlowのエコシステム／機能構成図」にあるように、TensorFlow 2.x時代のデータ管理機能である。

### リスト6-2　Feature columns（フィーチャーカラム）の作成

In [0]:
# 先ほどの列名を使って、フィーチャーカラムのリスト値を作成
my_feature_columns = []
for key in train_X.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

my_feature_columns
# [NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
#  NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]  ……などと表示される

Feature columns（フィーチャーカラム：特徴の列項目）は、ニューラルネットワークに入力する特徴データ（行列データ）における個々の列データを意味する。特徴は必ずしも連続的な数値とは限らない。具体的にはカテゴリ値、ハッシュ化値など他にも考えられる。フィーチャーカラムは、こういったさまざまな特徴データが取り扱える機能となっている。元々はtf.estimatorの機能であったが、tf.dataとともにTensorFlow 2.xの基本機能に取り込まれており、Kerasでも活用できる。

### リスト6-3　作成済みEstimatorsを利用した書き方

In [0]:
# 学習時のログ保存ディレクトリのパス名を定義
import datetime
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/estimator/' + current_time + '/train'

# ハイパーパラメーターとなる変数（モデル定義時に必要となる数値） 
# ### 活性化関数（隠れ層用）の定義 ###
activation = tf.nn.tanh # 活性化関数（隠れ層用）： tanh関数（変更可能）
                        # 全レイヤーで共通。指定しない場合は「tf.nn.relu」
# ### 活性化関数（出力用）は指定できない ###
# 回帰（DNNRegressor）の場合、線形（Linear）関数（＝確率値のまま）
# 分類（DNNClassifier）の場合、ソフトマックス（Softmax）関数

# ### 損失関数も指定できない ###
# 回帰（DNNRegressor）の場合、平均二乗誤差（Mean Squared Error）
# 分類（DNNClassifier）の場合、交差エントロピー誤差（Cross Entropy）

# ### 最適化アルゴリズムの定義 ###
optimizer = tf.keras.optimizers.SGD(learning_rate=0.03) # SGDで、学習率は0.03

# モデルの設計
model = tf.estimator.DNNRegressor(      # モデルの生成

    # ### レイヤーを定義 ###
    feature_columns=my_feature_columns, # 入力層
    hidden_units=[                      # 隠れ層
        LAYER1_NEURONS,  # 隠れ層：1つ目のレイヤー
        LAYER2_NEURONS,  # 隠れ層：2つ目のレイヤー
        OUTPUT_RESULTS],                # 出力層

    activation_fn=activation,  # 活性化関数 
                               # 損失関数は指定できない
    optimizer=optimizer,       # 最適化  
    model_dir=train_log_dir)   # 学習時のログ保存ディレクトリ

### ●学習と推論： 書き方入門（tf.estimator専用）

### リスト6-4　学習、精度検証（＝評価）するコード

In [0]:
# データの入力は「入力関数」として定義する必要がある
def input_fn(features_X, labels_y, training=True, batch_size=15):
  # 特徴データとラベルをtf.dataのデータセットとしてまとめる
  dataset = tf.data.Dataset.from_tensor_slices((dict(features_X), labels_y))

  # 訓練時のみ、データセットをシャッフルする
  if training:
      dataset = dataset.shuffle(250, reshuffle_each_iteration=True).repeat()
  # 精度検証（＝評価）時は、シャッフル不要
  
  # バッチサイズ分だけ返す
  return dataset.batch(batch_size)

# 学習前にログを空にする
!rm -rf ./logs/

# 学習する（＝モデルを訓練する）
model.train(
    input_fn=lambda: input_fn(train_X, train_y, training=True), # データ入力
    steps=int(250/15)*100) # データ件数「250」／バッチサイズ「15」×「100」エポック＝「1600」ステップ

# 評価する（＝モデルの精度を検証する）
model.evaluate(
    input_fn=lambda: input_fn(valid_X, valid_y, training=False))

`input_fn()`関数の`training`引数は、訓練時か精度検証（＝評価）時かで処理を分岐できるようにするために追加した独自のパラメーターである。引数`batch_size`はバッチサイズを指定できるようにしたもので、デフォルト値を**15**としている。
tf.dataのデータセットを作成する場合は、`tf.data.Dataset.from_tensor_slices`メソッドを呼び出せばよい。tf.dataのデータセット（Datasetオブジェクト）は、Pythonイテラブルであり、Pythonイテレーションを通じてエポックを作成することができるというメリットがある。

そのデータセットの`shuffle()`メソッドに指定している**250**は、バッファサイズである。バッファサイズは、データ件数以上を指定しないと完全にシャッフルできないので、今回はデータ件数である**250**を指定した。引数`reshuffle_each_iteration`はデータセットの末尾まで来たら再度シャッフルするかどうかを指定している。引数なしの`repeat()`メソッドにより、データセットは最後まで来ると再スタートして、無限に繰り返されるようになる。この書き方は、TensorFlow 1.x時代のtd.dataのエポック作成方法である。TensorFlow 2.x時代は`repeat()`せずにforループで回すのが一般的な書き方である。

生成したモデルの`train()`／`evaluate()`メソッドの引数`input_fn`には、先ほど作成した`input_fn()`関数を指定するわけだが、`input_fn()`関数そのものは引数によって挙動をカスタマイズできるようにしているので、ラムダ（lambda）による匿名関数を作って、それを引数`input_fn`に指定している。ラムダ関数の中は、学習時と精度評価時で引数を変えて`input_fn()`関数を呼び出している。こうすることでコードを書く量を減らしている。

### リスト6-5　テスト、推論（予測）するコード（共通）

In [0]:
def test_fn(features, batch_size=15):
  # ※テスト時はシャッフル不要
  return tf.data.Dataset.from_tensor_slices(features).batch(batch_size)

list(model.predict(input_fn=lambda: test_fn({ 'x': [0.1], 'y': [-0.2] })))

In [0]:
# 次に進む前にモデルをリセット
tf.keras.backend.clear_session() # グラフを破棄する
del model                        # 変数を削除する

# お疲れさまでした。『TensorFlow 2.0最新の書き方入門』は修了です。